In [31]:
import pandas as pd
import os
import subprocess
import re

In [ ]:
def get_columns_to_pause(df):
    pattern = re.compile(r'^pause\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [ ]:
def create_silent_audio(duration, output_file):
    # Command to create silent audio with ffmpeg
    command = [
        'ffmpeg',
        '-f', 'lavfi',
        '-i', 'anullsrc=channel_layout=stereo:sample_rate=48000',
        '-t', duration,
        '-c:a', 'aac',
        '-b:a', '191k',
        output_file
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Silent audio file {output_file} created successfully.")
        
    except subprocess.CalledProcessError as e:
        print(f"Error creating silent audio file: {e}")

In [16]:
def get_the_table_data():
    for f in os.listdir():
        if f.endswith(".csv"):
                video_schnitt_df = pd.read_csv(f)
                print(f"Table {f} loaded succesfully")
                return video_schnitt_df
    raise ValueError("There is no csv in this directory")

In [17]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [18]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'libx264', output_file # libx264 to encode and copy to test (but that didnt work the last time)
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# Get file and clean it

In [19]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)             

Table Videoschnitt_Liste - Spaltennamen (1).csv loaded succesfully
Table Columns cleaned


In [20]:
video_schnitt_df

,dateiname,notiz,rausschneiden1_ab,rausschneiden1_bis,rausschneiden2_ab,rausschneiden2_bis,rausschneiden3_ab,rausschneiden3_bis,rausschneiden4_ab,standbild_bis,teilen1_bei,teilen2_bei,teilen3_bei,teilen4_bei,pause1_bei,pause1_dauer,pause2_bei,pause2_dauer
0,Normalverteilung.mp4,nan,nan,nan,nan,nan,nan,nan,nan,00:01:00,00:01:30,00:02:40,00:04:00,00:04:30,nan,nan,nan,nan
1,ELR_Geradengleichung.mp4,nan,nan,00:02:30,00:03:00,nan,nan,nan,nan,nan,00:02:40,nan,nan,nan,nan,nan,nan,nan
2,ELR_Geradengleichung_Einleitung.mp4,nan,00:01:00,00:02:30,nan,nan,nan,nan,nan,00:00:30,00:03:00,00:04:00,nan,nan,nan,nan,nan,nan
3,Normalverteilung2.mp4,nan,00:01:00,00:02:30,00:03:00,00:03:30,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
# get timestamp pause1_bei
# create standbild
# get timestamp pause1_duration
# create silent audio for the duration
# merge audio with standbild
# add pause into the video at the right place

In [24]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'libx264', # needs to stay libx264
               '-c:a', 'copy', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [33]:
get_columns_to_pause(video_schnitt_df)

['pause1_bei', 'pause1_dauer', 'pause2_bei', 'pause2_dauer']

In [25]:
get_standbild("Normalverteilung.mp4","standbild.mp4","00:01:00")

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

In [27]:
create_silent_audio("00:00:30","audio.mp4")

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

Silent audio file audio.mp4 created successfully.


size=      15kB time=00:00:29.99 bitrate=   4.0kbits/s speed=51.8x    
video:0kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 75.638321%
[aac @ 0x7fceeb72b580] Qavg: 65536.000


In [28]:
merge_audio_and_video("standbild.mp4","audio.mp4","merged.mp4")

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl